# Categories Table

In [1]:
import sqlite3
from bs4 import BeautifulSoup
import requests

connection = sqlite3.connect('BooksDB.db')
sql = "CREATE TABLE categories (id int NOT NULL,category varchar(255),link varchar(255))"
cursor = connection.execute(sql)

cat_url = f'https://books.toscrape.com/index.html'
response = requests.get(cat_url)
soup = BeautifulSoup(response.text,'html.parser')
cat_ul= soup.find_all('div', attrs={'class':'side_categories'})  
cat_ul= cat_ul[0].find_all('a') 

for i in range(1,len(cat_ul)):
    sql = 'INSERT INTO categories (id,category,link) VALUES (?,?,?)'
    data = [i-1,cat_ul[i].get_text().strip(),cat_ul[i]['href']]
    cursor = connection.execute(sql,data)

connection.commit()
connection.close()
print("Categories gathered.") 

# Books Table

In [ ]:
import sqlite3
from bs4 import BeautifulSoup
import requests
connection = sqlite3.connect('BooksDB.db')
sql = "CREATE TABLE books (name varchar(255),category varchar(255),price float,rating int)"
cursor = connection.execute(sql)
print("Table created.")


print("Gathering Books ... .")
for i in range(1,len(cat_ul)):
    j=1
    page='index'
    while True:
        url = f'https://books.toscrape.com/catalogue/category/books/{cat_ul[i].get_text().strip().replace(" ","-").lower()}_{i+1}/{page}.html'
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')
        books = soup.find_all('article', attrs={'class':'product_pod'}) 
        for book in books: 
            book_name = book.find('h3').get_text()
            book_price = book.find('p',attrs={'class':'price_color'}).get_text()[2::]
            book_category = cat_ul[i].get_text().strip()
            book_rating = book.find('p',attrs={'class':['star-rating One','star-rating Two','star-rating Three','star-rating Four','star-rating Five']})['class'][1]
            
            if book_rating == 'One' : 
                book_rating = 1
            elif book_rating=='Two' :
                book_rating = 2
            elif book_rating=='Three' :
                book_rating = 3
            elif book_rating=='Four' :
                book_rating = 4
            elif book_rating=='Five' :
                book_rating = 5
                
            sql = 'INSERT INTO books(name,category,price,rating) VALUES (?,?,?,?)' 
            data = [book_name,book_category,float(book_price),int(book_rating)]
            cursor = connection.execute(sql,data) 
        query_pages = soup.find_all('li', attrs={'class':'next'})
        if len(query_pages)==0:
            break
        j+=1
        page = f'page-{j}'
connection.commit()
print("Books Gathered.")
connection.close()    

# Pandas

In [4]:
import pandas as pd
x= pd.read_csv('books.csv')
x

,name,category,price,rating
0,It's Only the Himalayas,Travel,45.17,2
1,Full Moon over Noahâs ...,Travel,49.43,4
2,See America: A Celebration ...,Travel,48.87,3
3,Vagabonding: An Uncommon Guide ...,Travel,36.94,2
4,Under the Tuscan Sun,Travel,37.33,3
...,...,...,...,...
995,Why the Right Went ...,Politics,52.65,4
996,Equal Is Unfair: America's ...,Politics,56.86,1
997,Amid the Chaos,Cultural,36.58,1
998,Dark Notes,Erotica,19.19,5


In [5]:
y= pd.read_csv('categories.csv')
y

,id,category,link
0,0,Travel,catalogue/category/books/travel_2/index.html
1,1,Mystery,catalogue/category/books/mystery_3/index.html
2,2,Historical Fiction,catalogue/category/books/historical-fiction_4/...
3,3,Sequential Art,catalogue/category/books/sequential-art_5/inde...
4,4,Classics,catalogue/category/books/classics_6/index.html
5,5,Philosophy,catalogue/category/books/philosophy_7/index.html
6,6,Romance,catalogue/category/books/romance_8/index.html
7,7,Womens Fiction,catalogue/category/books/womens-fiction_9/inde...
8,8,Fiction,catalogue/category/books/fiction_10/index.html
9,9,Childrens,catalogue/category/books/childrens_11/index.html
